# Results

Import packages.

In [31]:
import pandas as pd
from pathlib import Path

Specify the configuration to read results for.

In [32]:
# * Model choice
# alias = "SLSQPEnsemble_3-models_chronos-flowstate-timesfm_opt-mae_1-windows"

# * Random metric
alias = "SLSQPEnsemble_3-models_moirai-sundial-toto_opt-random_1-windows"

# * Additional models
# alias = "SLSQPEnsemble_4-models_chronos-moirai-sundial-toto_opt-mae_1-windows"

Read all of the configuration's CSV files into a single DataFrame and save the aggregated results.

In [33]:
input_path = Path(f"../results/{alias}")
csv_files = list(input_path.rglob("*.csv"))

df = pd.concat((pd.read_csv(f).tail(1) for f in csv_files), ignore_index=True)
print(f"df shape: {df.shape}")

output_path = Path(f"../results/all_results/{alias}") / "all_results.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)
print(f"Saved aggregatted results to {output_path}")

df.head()

df shape: (48, 15)
Saved aggregatted results to ../results/all_results/SLSQPEnsemble_3-models_moirai-sundial-toto_opt-random_1-windows/all_results.csv


,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,us_births/M/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,4.410284e+07,4.410284e+07,5426.967325,0.613907,0.016730,0.016926,4.249311,6640.997002,0.020627,0.016856,0.012764,Healthcare,1
1,us_births/W/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,1.670029e+06,1.670029e+06,990.722241,0.901148,0.013476,0.013526,6.990632,1292.295979,0.017543,0.013449,0.010871,Healthcare,1
2,us_births/D/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,1.672966e+05,1.672966e+05,242.106004,0.356472,0.023300,0.022961,3.736674,409.019051,0.038342,0.022695,0.018589,Healthcare,1
3,kdd_cup_2018/D/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,2.916049e+03,2.916049e+03,21.301093,1.203035,0.528329,0.468881,9.137324,54.000456,1.209668,0.477167,0.377479,Nature,1
4,kdd_cup_2018/H/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,4.075970e+03,4.075970e+03,19.263655,0.797581,0.754006,0.436380,10.047158,63.843326,1.336356,0.403223,0.342175,Nature,1


View the number of datasets for each term.

In [ ]:
num_short_datasets = df["dataset"].str.contains("short", case=False, na=False).sum()
print(f"Number of short datasets: {num_short_datasets}")

Number of short datasets: 48


View the number of datasets for each domain.

In [ ]:
domain_counts = df.groupby("domain").size().reset_index(name="num_evaluated_datasets")

display(domain_counts)

,domain,num_evaluated_datasets
0,Econ/Fin,3
1,Energy,14
2,Healthcare,5
3,Nature,8
4,Sales,4
5,Transport,7
6,Web/CloudOps,7


Display all of the results and only show the MASE and CRPS columns.

In [ ]:
pd.set_option("display.max_rows", None)

df = df.rename(
    columns={
        "eval_metrics/MASE[0.5]": "MASE",
        "eval_metrics/mean_weighted_sum_quantile_loss": "CRPS",
    },
    errors="ignore",
)

keep_cols = [
    "dataset",
    "MASE",
    "CRPS",
    "domain",
    "num_variates",
]
df = df[keep_cols].sort_values(by="CRPS", ascending=False).reset_index(drop=True)
display(df)

,dataset,MASE,CRPS,domain,num_variates
0,car_parts/M/short,0.815056,0.936593,Sales,1
1,hierarchical_sales/D/short,0.789432,0.649840,Sales,1
2,bitbrains_rnd/H/short,5.825690,0.633505,Web/CloudOps,2
3,bitbrains_fast_storage/H/short,1.001568,0.547587,Web/CloudOps,2
4,bitbrains_rnd/5T/short,1.651169,0.377719,Web/CloudOps,2
5,kdd_cup_2018/D/short,1.203035,0.377479,Nature,1
6,saugeen/W/short,1.177231,0.373321,Nature,1
7,hierarchical_sales/W/short,0.728154,0.348017,Sales,1
8,kdd_cup_2018/H/short,0.797581,0.342175,Nature,1
9,solar/10T/short,0.640838,0.341528,Energy,1
